In [1]:
import nbformat
import import_ipynb
import datacraft_danone as dd
import pandas as pd
import numpy as np

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import State,Output, Input

from dash import html
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate

importing Jupyter notebook from datacraft_danone.ipynb


WARNING - (py.warnings._showwarnmsg) - <string>:29: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`


WARNING - (py.warnings._showwarnmsg) - C:\Users\amiar\AppData\Local\Temp\ipykernel_2092\200530629.py:8: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`




In [2]:
df = pd.read_parquet("./data/forecasting.parquet.gzip")

In [3]:
params = {
    "changepoints" : None, #lorsqu'on donne ce paramètre, le model ne va detecter aucun changepoints (ce n'est pas le paramètre par défaut)
    "n_changepoints" : 25, # nombre de changepoints # pas utile si on spécifie changepoints
    "changepoints_range" : 0.8, # n_changepoints répartis sur 80% sur train set 
    "trend_reg" : 0,
    "trend_reg_threshold" : False,
    "yearly_seasonality" :"auto",
    "weekly_seasonality" :"auto",
    "daily_seasonality": "auto",
    "seasonality_mode" : "additive",
    "seasonality_reg" : 0,
    "n_forecasts" : 1,
    "n_lags" : 0,
    "num_hidden_layers" : 0,
    "d_hidden" : None,
    "ar_reg" : None,
    "learning_rate" : None,
    "epochs" : None,
    "batch_size" : None,
    "loss_func" : "Huber",
    "optimizer" : "AdamW",
    "newer_samples_weight" : 2,
    "newer_samples_start" : 0.0,
    "impute_missing" : True,
    "collect_metrics" :True,
    "normalize" :"auto",
    "global_normalization" :False,
    "global_time_normalization" : True,
    "unknown_data_normalization" : False,
}

In [8]:
def interactive_dashboard(danone, params):
    
    app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])
    app.layout = html.Div([
        html.H1('Analyse TS Danone avec Prophet'),

        dcc.Dropdown(id='dd_product', options=[{'value':k, 'label': k} for k in danone['product_index'].unique()], multi=True, value=[]),

        dcc.Dropdown(id='dd_year', options=[{'value': k, 'label': k} for k in np.sort(danone['time_index'].apply(lambda date:date.year).unique())], multi=True, value=[]),

        dcc.RadioItems(id='radio_holiday', options=[{'label': 'add holidays effect', 'value': True}, {'label': "don't add holidays effect", 'value': False}]),
        
        dcc.Dropdown(id="dd_periode", options=[{'value' : k, 'label': str(k) + "semaines"} for k in range(1,53)], multi=True, value=[]),

        dcc.Dropdown(id='dd_reg', options=[{'label': reg, 'value': reg} for reg in danone.columns], multi=True, value=[]),

        dbc.Button("apply changements", id='button'),

        dcc.Graph(id='graph_1')
    ])

    @app.callback(Output('graph_1', 'figure'),
                 Input('button', 'n_clicks'),
                 State('dd_product', 'value'),
                 State('dd_year', 'value'),
                 State('radio_holiday', 'value'),
                 State('dd_periode', 'value'),
                 State('dd_reg', 'value')
                 )

    def plot_graph1(n_clicks, product, year, holiday_bool, periode, reg):
        if not n_clicks:
            raise PreventUpdate
        else:
            print(product)
            m, df, future, forecast, residuals = dd.train_neural_prophet_model(
                df= danone,params = params, periode = periode[0], date_sep = year[0])
            m, df, future, forecast2, residuals = dd.train_neural_prophet_model(
                df= danone,params = params, periode = periode[0],date_sep=year[0]+1)
            
            #product_id=product, add_holiday=holiday_bool, regressor=reg, interval=0.95)

            return dd.plot_my_neural_graph(forecasts = [forecast, forecast2],
                                           product_id=product, year=year)

    app.run_server(mode='inline')

In [9]:
interactive_dashboard(df,params)

INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 99.048% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as W-MON
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 98.81% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_bat

[70189, 70193]


WARNING - (py.warnings._showwarnmsg) - D:\Users\amiar\anaconda3\lib\site-packages\neuralprophet\utils.py:205: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


WARNING - (py.warnings._showwarnmsg) - D:\Users\amiar\anaconda3\lib\site-packages\neuralprophet\utils.py:205: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


WARNING - (py.warnings._showwarnmsg) - D:\Users\amiar\anaconda3\lib\site-packages\neuralprophet\utils.py:205: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


WARNING - (py.warnings._showwarnmsg) - D:\Users\amiar\anaconda3\lib\site-packages\neuralprophet\utils.py:205: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


WARNING - (py.wa

  0%|          | 0/106 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.74E-02, min: 2.87E+00


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.74E-02, min: 6.31E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.69E-02
Epoch[602/602]: 100%|█| 602/602 [00:16<00:00, 35.59it/s, SmoothL1Loss=0.0121, MAE=43.8, RMSE=57.2, RegLoss=0, MAE_val=8
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE   RMSE
1         0.010 46.829 52.104
INFO - (NP

  0%|          | 0/108 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.24E-02, min: 1.14E-01


  0%|          | 0/108 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.72E-02, min: 9.58E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.40E-02
Epoch[469/469]: 100%|█| 469/469 [00:17<00:00, 26.87it/s, SmoothL1Loss=0.0117, MAE=42, RMSE=53.1, RegLoss=0, MAE_val=111
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.df_utils._infer_frequency) - Major frequency W-MON corresponds to 75.0% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W-MON
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss     MAE    RMSE
1         0.074 129.295 135.347
